In [18]:
using LinearAlgebra, Random, Statistics, FFTW

In [3]:
function λi(Nx, Δx)
    is = reshape(1:Nx, Nx, 1, 1)
    @. (2sin((is-1)*π/Nx) / Δx)^2
end

function λj(Ny, Δy)
    js = reshape(1:Ny, 1, Ny, 1)
    @. (2sin((js-1)*π/Ny) / Δy)^2
end

λj (generic function with 1 method)

In [4]:
"""
    tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
Solve the tridiagonal system of linear equations described by the tridiagonal
matrix `M` with right-hand-side `g` assuming one of the eigenvalues is zero
(which results in a singular matrix so the general Thomas algorithm has been
modified slightly).
Reference CPU implementation per Numerical Recipes, Press et. al 1992 (§ 2.4)
"""
function tridiag(M, f)
    N = length(f)
    ϕ = similar(f)
    γ = similar(f)

    β    = M.d[1]
    ϕ[1] = f[1] / β

    for j = 2:N
        γ[j] = M.du[j-1] / β
        β    = M.d[j] - M.dl[j-1] * γ[j]

        # This should only happen on last element of forward pass for problems
        # with zero eigenvalue. In that case the algorithmn is still stable.
        abs(β) < 1.0e-12 && break

        ϕ[j] = (f[j] - M.dl[j-1] * ϕ[j-1]) / β
    end

    for j = 1:N-1
        k = N-j
        ϕ[k] = ϕ[k] - γ[k+1] * ϕ[k+1]
    end

    return ϕ
end

tridiag

In [5]:
δ(k, ΔzF, ΔzC, kx², ky²) = - (1/ΔzF[k-1] + 1/ΔzF[k]) - ΔzC[k] * (kx² + ky²)

function solve_poisson_1d(Nz, ΔzF, ΔzC, kx², ky², F)
    ld = [1/ΔzF[k] for k in 1:Nz-1]
    ud = copy(ld)
    d = [-1/ΔzF[1], [δ(k, ΔzF, ΔzC, kx², ky²) for k in 2:Nz]...]    
    M = Tridiagonal(ld, d, ud)

    ϕ = tridiag(M, F)
    
    return ϕ
end

solve_poisson_1d (generic function with 1 method)

In [6]:
function grid(zF)
    Nz = length(zF) - 1
    ΔzF = [zF[k+1] - zF[k] for k in 1:Nz]
    zC = [(zF[k] + zF[k+1]) / 2 for k in 1:Nz]
    ΔzC = [zC[k+1] - zC[k] for k in 1:Nz-1]
    return zF, zC, ΔzF, ΔzC
end

grid (generic function with 1 method)

In [7]:
Nx, Ny = 4, 4
Lx, Ly = 1, 1
Δx, Δy = Lx/Nx, Ly/Ny

zF = [1, 2, 4, 7, 11, 16, 22, 29, 37]
Nz = length(zF) - 1
zF, zC, ΔzF, ΔzC = grid(zF)

ΔzC = [ΔzC..., ΔzC[end]]

@show zF, zC, ΔzF, ΔzC

zC = reshape(zC, (1, 1, Nz))
zF = reshape(zF, (1, 1, Nz+1))
ΔzC = reshape(ΔzC, (1, 1, Nz))
ΔzF = reshape(ΔzF, (1, 1, Nz));

(zF, zC, ΔzF, ΔzC) = ([1, 2, 4, 7, 11, 16, 22, 29, 37], [1.5, 3.0, 5.5, 9.0, 13.5, 19.0, 25.5, 33.0], [1, 2, 3, 4, 5, 6, 7, 8], [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 7.5])


In [8]:
R = rand(MersenneTwister(0), Nx, Ny, Nz)
R .= R .- mean(R)
mean(R)

7.28583859910259e-17

In [9]:
F = ΔzC .* R;

In [10]:
F̃ = fft(F, [1, 2])
ϕ̃ = similar(F̃)

kx² = λi(Nx, Δx)
ky² = λj(Ny, Δy)

for i in 1:Nx, j in 1:Ny
    ϕ̃[i, j, :] = solve_poisson_1d(Nz, ΔzF, ΔzC, kx²[i], ky²[j], F̃[i, j, :])
end

# ϕ̃[1, 1, 1] = 0
ϕ = real.(ifft(ϕ̃, [1, 2]))
ϕ = ϕ .- mean(ϕ)

4×4×8 Array{Float64,3}:
[:, :, 1] =
 0.234736  1.06724  0.948748   1.08669 
 0.101265  1.18547  0.0257168  0.111761
 1.22349   1.42561  0.610413   1.2229  
 1.20509   1.38001  0.6783     0.485393

[:, :, 2] =
 0.701851  0.717204  0.723133  0.708389
 0.698443  0.722331  0.717185  0.708474
 0.701978  0.720702  0.720774  0.705093
 0.702727  0.71406   0.718877  0.700209

[:, :, 3] =
 0.405554  0.397676  0.396754  0.408218
 0.404011  0.409989  0.400947  0.402341
 0.412444  0.40856   0.413709  0.413739
 0.400166  0.39743   0.408483  0.402269

[:, :, 4] =
 -0.226056  -0.235762  -0.234156  -0.224836
 -0.22607   -0.233716  -0.232831  -0.229138
 -0.229492  -0.237278  -0.238219  -0.236312
 -0.239724  -0.241894  -0.242626  -0.240596

[:, :, 5] =
 -0.589929  -0.593238  -0.589624  -0.591637
 -0.601183  -0.592465  -0.590204  -0.59394 
 -0.602116  -0.598396  -0.58716   -0.596824
 -0.595467  -0.594412  -0.591255  -0.596162

[:, :, 6] =
 -0.8843    -0.886481  -0.891237  -0.888228
 -0.902248  -0.900229  

In [11]:
@inline δx_caa(i, j, k, f) = @inbounds f[i+1, j, k] - f[i, j, k]
@inline δy_aca(i, j, k, f) = @inbounds f[i, j+1, k] - f[i, j, k]
@inline δz_aac(i, j, k, f) = @inbounds f[i, j, k+1] - f[i, j, k]

@inline ∂x_caa(i, j, k, Δx,  f) = δx_caa(i, j, k, f) / Δx
@inline ∂y_aca(i, j, k, Δy,  f) = δy_aca(i, j, k, f) / Δy
@inline ∂z_aac(i, j, k, ΔzF, f) = δz_aac(i, j, k, f) / ΔzF[k]

@inline ∂x²(i, j, k, Δx, f)       = (∂x_caa(i, j, k, Δx, f)  - ∂x_caa(i-1, j, k, Δx, f))  / Δx
@inline ∂y²(i, j, k, Δy, f)       = (∂y_aca(i, j, k, Δy, f)  - ∂y_aca(i, j-1, k, Δy, f))  / Δy
@inline ∂z²(i, j, k, ΔzF, ΔzC, f) = (∂z_aac(i, j, k, ΔzF, f) - ∂z_aac(i, j, k-1, ΔzF, f)) / ΔzC[k]

@inline ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, f) = ∂x²(i, j, k, Δx, f) + ∂y²(i, j, k, Δy, f) + ∂z²(i, j, k, ΔzF, ΔzC, f)

∇² (generic function with 1 method)

In [12]:
∇²ϕ = zeros(Nx, Ny, Nz)

# ∇²ϕ[1] = ∂z_aac(1, ΔzF, ϕ) / ΔzC[1]
for i in 1:Nx, j in 1:Ny, k in 2:Nz-1
    ∇²ϕ[i, j, k] = ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, ϕ)
end
# ∇²ϕ[Nz] = (∂z_aac(Nz, ΔzF, ϕ) - ∂z_aac(Nz-1, ΔzF, ϕ)) / ΔzC[Nz-1]
∇²ϕ

4×4×8 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
  2.63915  -0.224261  -0.546163  -4.349  
 -9.19137  -0.44987   -0.244253  -5.0874 
  8.7404   -0.159109  -0.445391  -4.30666
 -2.93595   0.350236  -0.59151   -9.28387

[:, :, 3] =
 9.39127   0.333454    0.262613  -10.439  
 5.10874  -0.476893    0.288881   -9.81686
 4.26906  -0.0275773  -0.387915  -10.6784 
 4.89583   0.37221    -0.214547  -20.0608 

[:, :, 4] =
 20.0726   0.177416    0.0478176   -6.31757
  9.90356  0.0744746  -0.0353552   -6.03128
 10.085    0.120458    0.0901368   -5.80898
 10.4919   0.248457    0.428237   -11.2034 

[:, :, 5] =
 11.2036    0.0930314  -0.170691   -4.67528
  6.26345  -0.205434   -0.0328814  -4.87683
  6.03967   0.284126   -0.444014   -4.68678
  5.62967   0.0514495  -0.0644843  -9.19768

[:, :, 6] =
 8.989    -0.258628  -0.0604299   5.67851
 5.25501   0.169956   0.0842943   6.08865
 5.15037   0.417122  -0.0329769   6.54538
 

In [13]:
R

4×4×8 Array{Float64,3}:
[:, :, 1] =
  0.31141   -0.233358  -0.150409   -0.252202
  0.398119  -0.308761   0.460979    0.397809
 -0.347672  -0.469936   0.0735737  -0.345202
 -0.334909  -0.443969   0.0270514   0.143211

[:, :, 2] =
 0.0636496  -0.0429341  -0.469097   0.0930589
 0.356041   -0.44987    -0.244253  -0.376493 
 0.455562   -0.159109   -0.445391   0.32588  
 0.255452    0.255364   -0.355601   0.402474 

[:, :, 3] =
 -0.212163   0.289686    0.439453  -0.432683
  0.210612  -0.476893    0.288881   0.264436
 -0.392585  -0.0275773  -0.387915  -0.407415
  0.254832   0.386961   -0.397969   0.325837

[:, :, 4] =
 -0.328123  0.142684    0.0361043  -0.463116 
 -0.200093  0.0744746  -0.0353552  -0.0294764
 -0.31583   0.120458    0.0901368   0.153014 
  0.361343  0.222766    0.279108    0.330018 

[:, :, 5] =
 -0.343131     0.109918    -0.120178   -0.0438408
  0.426619    -0.205434    -0.0328814  -0.0597812
  0.271488     0.284126    -0.444014    0.131973 
 -0.00643321  -0.00636371  -0.0322

In [14]:
∇²ϕ .≈ R

4×4×8 BitArray{3}:
[:, :, 1] =
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

[:, :, 2] =
 0  0  0  0
 0  1  1  0
 0  1  1  0
 0  0  0  0

[:, :, 3] =
 0  0  0  0
 0  1  1  0
 0  1  1  0
 0  0  0  0

[:, :, 4] =
 0  0  0  0
 0  1  1  0
 0  1  1  0
 0  0  0  0

[:, :, 5] =
 0  0  0  0
 0  1  1  0
 0  1  1  0
 0  0  0  0

[:, :, 6] =
 0  0  0  0
 0  1  1  0
 0  1  1  0
 0  0  0  0

[:, :, 7] =
 0  0  0  0
 0  1  1  0
 0  1  1  0
 0  0  0  0

[:, :, 8] =
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

In [17]:
@test ∇²ϕ[2:Nx-1, 2:Ny-1, 2:Nz-1] ≈ R[2:Nx-1, 2:Ny-1, 2:Nz-1]

LoadError: UndefVarError: @test not defined